Référence(s):

> 


    https://docs.python.org/fr/3/library/_thread.html

> 


    http://effbot.org/zone/thread-synchronization.htm

> 



# Membres:


> Kévin Ushaka 


> Denis ADANLEHOUSSI






# Synchronizing

### Lock

Un verrou primitif est soit « verrouillé » soit « déverrouillé ». Il est créé dans un état déverrouillé. Il a deux méthodes, acquire() et release(). Lorsque l'état d'un verrou est déverrouillé, l'appel à acquire() le verrouille. Lorsque l'état d'un verrou est verrouillé, l'appel à acquire() le bloque jusqu'à ce qu'un appel à release() le déverrouille.

Lorsque plusieurs threads sont bloqués à cause de l'appel à acquire() par un autre thread en attendant que l'état passe à déverrouillé, un seul  de ces threads se poursuit lors de l'appel de release () par le thread à lorigine du verrouillage. Le thread parmis les threads en attente qui continue son execution (prend le verrou) n'est pas défini et peut varier selon les implémentations.

Pour un fonctionnement correct, il est important de libérer le verrou même si quelque chose ne va pas lors de l’accès à la ressource. On peut utiliser try-finally.

**Pseudo code :**


`lock.acquire()`

`try:`

    ... access shared resource

`finally:`

    lock.release() # release lock, no matter what


La méthode acquire() prend un indicateur d'attente facultatif, qui peut être utilisé pour éviter le blocage si le verrou est détenu par quelqu'un d'autre. Si vous passez False, la méthode ne bloque jamais, mais retourne False si le verrou était déjà maintenu.




**Preuve** : 


*Version 1 :* 

Dans l'exemple suivant, deux methodes utilisent une même varible globale "g", on gère pas les sections critqiues.

Dans `add_one_one()` on reinitialise g à 0 puis on l'incrément de 1. Par la suite on cré une boucle de 30 avec affichage de texte juste pour faire durer l'execution de la methode . On incremente encore une fois g avant de l'afficher
La resultat attendu appres cette afficheage est donc 2.

Dans `add_two()` on incremente juste g de 2.

Nous appelons deux threads sur ces deux methodes
(`add_one_one()` en premier et `add_two()` apres).

Notons que l'on attends que les deux threads se termine ( fonction join()), pour afficher le résultat seulement lexecution totale des deux Threads.

In [103]:
#lock_tut.py
from threading import Lock, Thread
lock = Lock()
g = 0

def add_one_one():
   """
   Just used for demonstration. It's bad to use the 'global'
   statement in general.
   """
   
   global g
   g=0
   g += 1
   for i in range(30):
     print('boucle d\'attente')
   g += 1
   print('Affichage de g dans add one one :',g)

def add_two():
   global g
   g += 2

threads = []
for func in [add_one_one, add_two]:
   threads.append(Thread(target=func))

threads[0].start() 
threads[1].start()

for thread in threads:
   """
   Waits for threads to complete before moving on with the main
   script.
   """
   thread.join()

print(g)

boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
Affichage de g dans add one one : 4
4


On remarque apres plusieurs executions que l'affichage de g dans add_one_one donne tantôt 2 tantôt 4 ce qui n'est pas normal. La variable g a donc été modifier en tre temps par le thread `add_two()` avant sont affichage dans `add_one_one()`.





*Version 2 :*

Dans cette seconde version, nous gèrons les sections critqiues avec le verrou threading.Lock

In [104]:
#lock_tut.py
from threading import Lock, Thread
lock = Lock()
g = 0

def add_one():
   """
   Just used for demonstration. It's bad to use the 'global'
   statement in general.
   """
   
   global g
   lock.acquire()
   g=0
   g += 1
   for i in range(30):
     print('boucle d\'attente')
   g += 1
   print('Affichage de g dans add one :',g)
   lock.release()

def add_two():
   global g
   lock.acquire()
   g += 2
   lock.release()

threads = []
for func in [add_one, add_two]:
   threads.append(Thread(target=func))
   threads[-1].start()

for thread in threads:
   """
   Waits for threads to complete before moving on with the main
   script.
   """
   thread.join()

print(g)

boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
boucle d'attente
Affichage de g dans add one : 2
4


On obtient toujours 2 pour l'affichage dans add_one_one et 4 a la fin, ce qui veut dire que les sections critiques ont été bien gérée.

### ReLock

Les objets de verrouillage normal ne peuvent pas être acquis plus d'une fois, même par le même thread. Cela peut introduire des effets secondaires indésirables si un verrou est accédé par plus d'une fonction dans la même chaîne d'appels.

In [114]:
import threading

lock = threading.Lock()

print('First try :',lock.acquire())
print('Second try:',lock.acquire(0))
print("print this if not blocked...")

First try : True
Second try: False
print this if not blocked...


Comme nous pouvons le voir dans le code, puisque les deux fonctions utilisent le même verrou global et que l'une appelle l'autre, la deuxième acquisition échoue et se serait bloquée en utilisant les arguments par défaut pour acquérir (blocking = True, timeout = -1) comme indiqué dans le code suivant et sa sortie :

In [123]:
import threading

lock = threading.Lock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(blocking = True, timeout = -1))

print("print this if not blocked...")

First try : True


KeyboardInterrupt: ignored

On remarque qu'on a vraiment un deadLock.

Ainsi, comme dans l'exemple ci-dessus, dans une situation où un code séparé du même thread doit «réacquérir» le verrou, nous devons utiliser un threading.RLock au lieu d'un simple threading.Lock () :

In [168]:
import threading

lock = threading.RLock()

print('First try :', lock.acquire())
print('Second try:', lock.acquire(blocking = True, timeout = -1))
print("print this if not blocked...")

First try : True
Second try: True
print this if not blocked...


### Semaphore

Les sémaphores sont simplement des compteurs avancés. Un `acquire()` appel à un sémaphore ne se bloquera qu'après qu'un certain nombre de threads l'auront `acquire()`. Le compteur associé diminue par appel à `acquire()` et augmente par appel à `release()`. Un `ValueError` se produira si les appels à `release()` tentent d'incrémenter le compteur au-delà de sa valeur maximale assignée (qui est le nombre de threads qui peuvent `acquire()` le sémaphore avant le blocage).Dans de nombreux cas, nous souhaitons autoriser plusieurs utilisateurs à accéder à une ressource tout en limitant le nombre total d'accès.

Par exemple, nous pouvons souhaiter utiliser un sémaphore dans une situation où nous devons prendre en charge des connexions / téléchargements simultanés. Les sémaphores sont également souvent utilisés pour protéger les ressources dont la capacité est limitée, par exemple un serveur de base de données. Le code suivant illustre l'utilisation de sémaphores:

In [163]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)

class ThreadPool(object):
    def __init__(self):
        super(ThreadPool, self).__init__()
        self.active = []
        self.lock = threading.Lock()
    def makeActive(self, name):
        with self.lock:
            self.active.append(name)
            logging.debug('Running: %s', self.active)
    def makeInactive(self, name):
        with self.lock:
            self.active.remove(name)
            logging.debug('Running: %s', self.active)

def f(s, pool):
    logging.debug('Waiting to join the pool')
    with s:
        name = threading.currentThread().getName()
        pool.makeActive(name)
        time.sleep(0.5)
        pool.makeInactive(name)


pool = ThreadPool()
s = threading.Semaphore(3)
for i in range(10):
  t = threading.Thread(target=f, name='thread_'+str(i), args=(s, pool))
  t.start()

(thread_0 ) Waiting to join the pool
(thread_0 ) Running: ['thread_0']
(thread_1 ) Waiting to join the pool
(thread_1 ) Running: ['thread_0', 'thread_1']
(thread_2 ) Waiting to join the pool
(thread_2 ) Running: ['thread_0', 'thread_1', 'thread_2']
(thread_3 ) Waiting to join the pool
(thread_4 ) Waiting to join the pool
(thread_5 ) Waiting to join the pool
(thread_6 ) Waiting to join the pool
(thread_7 ) Waiting to join the pool
(thread_8 ) Waiting to join the pool
(thread_9 ) Waiting to join the pool
(thread_6 ) Running: ['thread_7', 'thread_8']
(thread_9 ) Running: ['thread_7', 'thread_8', 'thread_9']
(thread_7 ) Running: ['thread_8', 'thread_9']
(thread_8 ) Running: ['thread_9']


Dans le code, la classe `ThreadPool` suit quels threads peuvent s'exécuter à un moment donné. Un vrai pool de ressources allouerait une connexion ou une autre valeur au thread nouvellement actif, et récupérerait la valeur lorsque le thread est terminé. Ici, il est utilisé uniquement pour contenir les noms des threads actifs pour montrer que seuls 10 sont exécutés simultanément.

## Objet Event

Nous utilisons plusieurs threads pour faire tourner des opérations séparées afin de les exécuter simultanément, cependant, il y a des moments où il est important de pouvoir synchroniser les opérations de deux ou plusieurs threads. L'utilisation d' objets Event est le moyen simple de communiquer entre les threads.

Un événement gère un indicateur interne que les appelants peuvent définir () ou effacer () . D'autres threads peuvent attendre () que l'indicateur soit défini () . Notez que la méthode wait () se bloque jusqu'à ce que l'indicateur soit vrai.

In [169]:
import threading
import time
import logging

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-9s) %(message)s',)
                    
def wait_for_event(e):
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    while not e.isSet():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other things')

if __name__ == '__main__':
    e = threading.Event()
    t1 = threading.Thread(name='blocking', 
                      target=wait_for_event,
                      args=(e,))
    t1.start()

    t2 = threading.Thread(name='non-blocking', 
                      target=wait_for_event_timeout, 
                      args=(e, 2))
    t2.start()

    logging.debug('Waiting before calling Event.set()')
    time.sleep(3)
    e.set()
    logging.debug('Event is set')

(blocking ) wait_for_event starting
(non-blocking) wait_for_event_timeout starting
(MainThread) Waiting before calling Event.set()
(non-blocking) event set: False
(non-blocking) doing other things
(non-blocking) wait_for_event_timeout starting
(MainThread) Event is set
(blocking ) event set: True
(non-blocking) event set: True
(non-blocking) processing event


# Barrier

Une barrière est une simple primitive de synchronisation qui peut être utilisée par différents threads pour s'attendre les uns les autres. Chaque thread tente de passer une barrière en appelant la wait()méthode, qui se bloquera jusqu'à ce que tous les threads aient effectué cet appel. Dès que cela se produit, les threads sont libérés simultanément. L'extrait de code suivant illustre l'utilisation de Barrier

In [170]:
#barrier_tut.py
from random import randrange
from threading import Barrier, Thread
from time import ctime, sleep

num = 4
# 4 threads will need to pass this barrier to get released.
b = Barrier(num)
names = ["Harsh", "Lokesh", "George", "Iqbal"]

def player():
    name = names.pop()
    sleep(randrange(2, 5))
    print("%s reached the barrier at: %s" % (name, ctime()))
    b.wait()
    
threads = []
print("Race starts now…")

for i in range(num):
    threads.append(Thread(target=player))
    threads[-1].start()
"""
Following loop enables waiting for the threads to complete before moving on with the main script.
"""
for thread in threads:
    thread.join()
print()
print("Race over!")

Race starts now…
George reached the barrier at: Thu Oct  1 21:16:17 2020
Iqbal reached the barrier at: Thu Oct  1 21:16:18 2020
Harsh reached the barrier at: Thu Oct  1 21:16:18 2020
Lokesh reached the barrier at: Thu Oct  1 21:16:19 2020

Race over!
